In [ ]:
# Connor Murray
# Started 12.1.2024; modified 2.26.2025
# Analyzing overlap of disease eQTLs and GTEx across tissue types

# Import libraries
import os
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import pyreadr as pr
import time
import matplotlib.pyplot as plt
import seaborn as sns

# Working directory
os.chdir("/standard/vol185/cphg_Manichaikul/users/csm6hg")

# GTEx eQTLs V10 list of Parquet files
gtex_files = [os.path.join("data/gtex_full_ciseqtl_v10/", f) 
              for f in os.listdir("data/gtex_full_ciseqtl_v10/") 
              if f.endswith(".parquet")]

# Find indices of files containing: heart
heart_indices = [i for i, file in enumerate(gtex_files) if "heart" in file.lower()]
heart_files = [gtex_files[i] for i in heart_indices]
print(heart_indices)

# TOPCHEF eQTLs - mapped using tensorqtl
chef_df = pr.read_r("nextflow_dna/output/analyses/qtl.rna.saturation.rds")
chef_df = chef_df[None]
chef_df = chef_df[chef_df['maxPC'] == 70]
chef_df = chef_df[chef_df["pval_perm"] < 0.05] # Restrict to best model

# Define a function for processing a single file
def process_file(file, output_file):
    start_time = time.time()
    
    # Load only the `variant_id` column
    gtex_df = pq.read_table(file, columns=['variant_id']).to_pandas()
    gtex_df = gtex_df.assign(
        chrom=gtex_df['variant_id'].str.split("_").str[0],
        position=gtex_df['variant_id'].str.split("_").str[1],
        snp=lambda x: x['chrom'] + ":" + x['position'])
    
    # Calculate proportion of variant IDs in GTEx SNPs
    prop_table = chef_df['variant_id'].isin(gtex_df['snp'])
    proportion = np.round(prop_table.value_counts(normalize=True)*100, 2)

    # Ensure both True and False proportions are present
    true_prop = proportion.get(True, 0.0)
    false_prop = proportion.get(False, 0.0)
    
     # Message
    print(file, true_prop)
    elapsed_time = time.time() - start_time
    print(f"Processed {os.path.basename(file)} in {elapsed_time:.2f} seconds.")
    
    # Write result to the output file
    with open(output_file, "a") as f:
        f.write(f"{os.path.basename(file)},{true_prop},{false_prop}\n")

    return {"file": os.path.basename(file), "snp_overlap_true": true_prop, "snp_overlap_false": false_prop}

# Output file
output_file = "/standard/vol185/cphg_Manichaikul/users/csm6hg/data/overlapGTEX_eqtl_results.csv"

# Write header to the output file
with open(output_file, "w") as f:
    f.write("file,true,false\n")

# Process files sequentially
results = []
for file in gtex_files:
    results.append(process_file(file, output_file))

# Convert results to a DataFrame
proportions_df = pd.DataFrame(results)

# Print the DataFrame
print(proportions_df.head())

# Output file
output_file = "/standard/vol185/cphg_Manichaikul/users/csm6hg/data/overlapGTEX_eqtl_results.csv"

# Read in overlap results
dt = pd.read_csv(output_file)

# Simplify name of tissue
dt['simp'] = dt['file'].str.replace('.v10.eQTLs.signif_pairs.parquet', '')

# Sort
dt=dt.sort_values('true', ascending=True)

# Create a stacked bar plot
plt.figure(figsize=(8, 16))

# Add bars for the "false" proportion
plt.barh(dt["simp"], dt["true"], color="lightblue")

# Formatting the plot
plt.title("TOPChef Overlap with GTEx v10 eQTLs")
plt.xlabel("Percentage", fontsize=16)
plt.ylabel("GTEx File", fontsize=16)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
#plt.tight_layout()

# Show the plot
plt.show()

(gtex_df['min_pval_nominal'] < gtex_df['pval_nominal_threshold']).value_counts()

# Double checking FDR is working - issue is that I do not have the full data and am just correcting significant data
from statsmodels.stats.multitest import fdrcorrection

# Filter rows where 'chrom' == "chr1"
chrom1_df = gtex_df[gtex_df['chrom'] == "chr1"]

# Perform FDR correction on p-values
pvals = chrom1_df['pval_nominal']
rejected, corrected_pvals = fdrcorrection(pvals)

# Filter rows where corrected p-values are less than 0.000377
significant_rows = chrom1_df[corrected_pvals < 0.000377]

# Display significant rows
print(significant_rows)